In [15]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from skimage.feature import local_binary_pattern
from skimage.color import rgb2lab, rgb2hsv
import pywt
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
from scipy.stats import spearmanr, pearsonr
from scipy import ndimage
from skimage.feature import hog, graycomatrix, graycoprops
from sklearn.tree            import DecisionTreeRegressor
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)


In [16]:
# Replace 'your_file.xlsx' with the path to your Excel file
file_path = "../../dataset/ratings/all_ratings_MOS.csv"

df = pd.read_csv(file_path)
df.head()


,filename,MOS
0,f1.png,47.058940
1,f10.png,59.343239
2,f100.png,67.845299
3,f101.png,66.061042
4,f102.png,55.699696


In [17]:
# Create bins for stratification
n_bins = 10  # You can adjust this number
df["Rating_bin"] = pd.qcut(df["MOS"], n_bins, labels=False)
df.head()


,filename,MOS,Rating_bin
0,f1.png,47.058940,1
1,f10.png,59.343239,2
2,f100.png,67.845299,4
3,f101.png,66.061042,4
4,f102.png,55.699696,2


In [18]:
# # Split into train (70%), val (15%), test (15%)
# train_df, temp_df = train_test_split(img_ratings, test_size=0.30, random_state=42)
# val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
# # print(train_df, val_df, test_df)

# Stratified split
train_df, temp_df = train_test_split(
    df, test_size=0.30, random_state=42, stratify=df["Rating_bin"]
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.50, random_state=42, stratify=temp_df["Rating_bin"]
)

# Stratified split sanity (Disjoint splits)
train_ids = set(train_df.index)
val_ids = set(val_df.index)
test_ids = set(test_df.index)
assert (
    train_ids.isdisjoint(val_ids)
    and train_ids.isdisjoint(test_ids)
    and val_ids.isdisjoint(test_ids)
), "Overlap between splits!"


# Compute mean & std on the TRAIN set only
mos_mean = train_df["MOS"].mean()
mos_std = train_df["MOS"].std()

# Apply z-score normalization to all three splits
for df_split in (train_df, val_df, test_df):
    df_split["MOS"] = (df_split["MOS"] - mos_mean) / mos_std


In [19]:
train_df.head()


,filename,MOS,Rating_bin
255,f329.png,0.134646,4
241,f316.png,-0.954147,1
175,f257.png,0.978417,9
119,f206.png,-3.041686,0
399,f459.png,-0.192770,3


In [20]:
val_df.head()


,filename,MOS,Rating_bin
445,f68.png,-0.149500,3
102,f191.png,-0.241829,3
198,f278.png,0.154798,4
439,f62.png,0.469404,5
72,f164.png,0.634564,6


In [21]:
test_df.head()


,filename,MOS,Rating_bin
161,f244.png,0.681995,7
30,f126.png,0.709697,7
523,r3.png,1.223393,9
51,f145.png,-1.211034,1
79,f170.png,-0.696548,2


In [22]:
X_train = list(train_df["filename"])
y_train = list(train_df["MOS"])

X_valid = list(val_df["filename"])
y_valid = list(val_df["MOS"])

X_test = list(test_df["filename"])
y_test = list(test_df["MOS"])


In [23]:
class HandcraftedRealnessPredictor:
    def __init__(self):
        
        self.model = DecisionTreeRegressor(max_depth=5, random_state=SEED)

    def extract_features(self, image):
        """
        Extracts 45 handcrafted features from RGB image (no resizing)
        """

        features = []
        image = np.asarray(Image.open("../../dataset/images/all-images/" + image))
        gray = np.mean(image, axis=2)
        h, w = gray.shape

        # 1. Enhanced Color Features (8 features)
        lab = rgb2lab(image)
        hsv = rgb2hsv(image)
        # Normalize LAB channels properly
        lab_a = lab[:, :, 1].astype(np.float32) / 127  # Green-red [-1,1]
        lab_b = lab[:, :, 2].astype(np.float32) / 127  # Blue-yellow [-1,1]

        features.extend(
            [
                np.std(lab_a),
                np.std(lab_b),  # Chroma variation
                hsv[:, :, 0].std(),  # Hue diversity
                np.percentile(hsv[:, :, 1], 95),  # Max saturation
                np.mean(np.abs(lab_a)),  # Green-red abnormality [0,1]
                np.mean(np.abs(lab_b)),  # Blue-yellow abnormality [0,1]
                np.median(hsv[:, :, 2]),  # Brightness median
                (hsv[:, :, 2] > 0.9).mean(),  # Overexposure ratio
            ]
        )



        # 2. Advanced Edge Features (10 features)
        sobel_x = ndimage.sobel(gray, axis=1)
        sobel_y = ndimage.sobel(gray, axis=0)
        edge_magnitude = np.hypot(sobel_x, sobel_y)

        features.extend(
            [
                edge_magnitude.mean(),
                edge_magnitude.std(),
                np.percentile(edge_magnitude, 90),  # Strong edge presence
                (edge_magnitude > 0.2).mean(),  # Edge density
                # Edge regularity
                np.corrcoef(sobel_x[::10].flatten(), sobel_y[::10].flatten())[0, 1],
                # Edge direction consistency
                np.std(np.arctan2(sobel_y, sobel_x)[edge_magnitude > 0.1]),
                # Asymmetry (AI artifacts often symmetric)
                np.abs(sobel_x - np.fliplr(sobel_x)).mean(),
                np.abs(sobel_y - np.flipud(sobel_y)).mean(),
                # Edge sharpness
                np.mean(edge_magnitude[edge_magnitude > 0.05]),
                np.max(edge_magnitude),
            ]
        )

        # 3. Texture & Pattern Features (15 features)
        glcm = graycomatrix(
            (gray * 255).astype(np.uint8),
            distances=[1, 3],
            angles=[0, np.pi / 4],
            levels=256,
            symmetric=True,
        )
        props = ["contrast", "dissimilarity", "homogeneity", "energy", "correlation"]
        for prop in props:
            features.extend(graycoprops(glcm, prop).flatten())

        # HOG features (captures structural patterns)
        hog_feats = hog(
            gray,
            orientations=8,
            pixels_per_cell=(32, 32),
            cells_per_block=(1, 1),
            visualize=False,
        )
        features.extend([hog_feats.mean(), hog_feats.std()])



        # 4. Frequency & Compression Artifacts (12 features)
        coeffs = pywt.wavedec2(gray, "db2", level=3)
        for i, (cH, cV, cD) in enumerate(coeffs[1:]):  # Skip approximation
            features.extend(
                [
                    np.abs(cH).mean(),
                    np.abs(cV).mean(),
                    np.abs(cD).mean(),
                    np.percentile(np.abs(cH), 90),
                ]
            )


        
        # 5. Perceptual Grouping Features (for psychological realism)
        # a) Contrast energy
        contrast_energy = ndimage.gaussian_filter(
            gray, sigma=1
        ) - ndimage.gaussian_filter(gray, sigma=3)
        features.append(np.mean(contrast_energy**2))

        # b) Visual saliency variance
        saliency = np.abs(ndimage.gaussian_gradient_magnitude(gray, sigma=2))
        features.extend([saliency.mean(), saliency.std()])

        return np.array(features)

    def fit(self, X, y):
        """X: List of RGB images, y: Realness scores"""
        features = np.array([self.extract_features(img) for img in X])
        self.model.fit(features, y)
        return self

    def predict(self, X):
        features = np.array([self.extract_features(img) for img in X])
        return self.model.predict(features)


### Initialize and train


In [24]:
predictor = HandcraftedRealnessPredictor()
predictor.fit(X_train, y_train)


#### predict on val


In [25]:
predictions = predictor.predict(X_valid)

score_r, _ = spearmanr(predictions, y_valid)
score_p, _ = pearsonr(predictions, y_valid)


print(score_r, score_p)


0.5504170661969076 0.5073133014002407


#### predict on test


In [26]:
predictions = predictor.predict(X_test)

score_r, _ = spearmanr(predictions, y_test)
score_p, _ = pearsonr(predictions, y_test)


print(score_r, score_p)


0.49781117350971243 0.43373095406300904
